In [32]:
import pandas as pd
from bs4 import BeautifulSoup
import requests

In [33]:
link = "https://www.goodreads.com/shelf/show/childrens"

In [34]:
response = requests.get(link)
response.status_code 

403

In [35]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}
response = requests.get(link, headers=headers)
response

<Response [200]>

In [6]:
soup = BeautifulSoup(response.text, "html.parser")

In [16]:
titles = []
for li in soup.select("a.leftAlignedImage[title]"):
    titles.append(li["title"])

titles


['Charlotte’s Web',
 'Harry Potter and the Philosopher’s Stone (Harry Potter, #1)',
 'Where the Wild Things Are',
 'The Giving Tree',
 'The Lion, the Witch and the Wardrobe (Chronicles of Narnia, #1)',
 'Matilda',
 'The Little Prince',
 'Charlie and the Chocolate Factory (Charlie Bucket, #1)',
 'Green Eggs and Ham',
 'Harry Potter and the Chamber of Secrets (Harry Potter, #2)',
 'Harry Potter and the Prisoner of Azkaban (Harry Potter, #3)',
 'The Secret Garden',
 'Where the Sidewalk Ends',
 'The Cat in the Hat (Cat in the Hat, #1)',
 'Harry Potter and the Goblet of Fire (Harry Potter, #4)',
 'James and the Giant Peach',
 'Harry Potter and the Order of the Phoenix (Harry Potter, #5)',
 'The Very Hungry Caterpillar',
 'Harry Potter and the Half-Blood Prince (Harry Potter, #6)',
 'A Wrinkle in Time (Time Quintet, #1)',
 'Harry Potter and the Deathly Hallows (Harry Potter, #7)',
 'The BFG',
 'Winnie-the-Pooh (Winnie-the-Pooh, #1)',
 'Anne of Green Gables (Anne of Green Gables, #1)',
 'Good

In [17]:
books = []

for a_tag in soup.select("a.leftAlignedImage[title]"):
    title = a_tag["title"].strip()
    link = BASE_URL + a_tag["href"]

    parent_div = a_tag.find_parent("div", class_="elementList")
    author_tag = parent_div.select_one("a.authorName span") if parent_div else None
    rating_tag = parent_div.select_one("span.greyText.smallText") if parent_div else None

    author = author_tag.text.strip() if author_tag else None
    rating, pub_year = None, None

    if rating_tag:
        text = rating_tag.text.strip()
        # Extract average rating
        if "avg rating" in text:
            try:
                rating = float(text.split("avg rating")[1].split("—")[0].strip())
            except:
                pass
        # Extract publication year (4 digits after 'published')
        if "published" in text:
            for word in text.split():
                if word.isdigit() and len(word) == 4:
                    pub_year = word
                    break

    books.append({
        "Title": title,
        "Author": author,
        "Rating": rating,
        "Publication Year": pub_year,
        "Link": link
    })

df = pd.DataFrame(books)
print(df.head(10))


                                               Title  \
0                                    Charlotte’s Web   
1  Harry Potter and the Philosopher’s Stone (Harr...   
2                          Where the Wild Things Are   
3                                    The Giving Tree   
4  The Lion, the Witch and the Wardrobe (Chronicl...   
5                                            Matilda   
6                                  The Little Prince   
7  Charlie and the Chocolate Factory (Charlie Buc...   
8                                 Green Eggs and Ham   
9  Harry Potter and the Chamber of Secrets (Harry...   

                     Author  Rating Publication Year  \
0                E.B. White    4.21             1952   
1              J.K. Rowling    4.47             1997   
2            Maurice Sendak    4.25             1963   
3          Shel Silverstein    4.38             1964   
4                C.S. Lewis    4.24             1950   
5                Roald Dahl    4.33            